In [122]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
pd.options.display.max_columns = 1000
import warnings
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)
from sklearn.model_selection import train_test_split
import xgboost as xgb
import random
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [123]:
# Load each data file into a data frame
item_categories = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/item_categories.csv", sep=",")
items = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/items.csv", sep=",")
shops = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/shops.csv", sep=",")
test = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/test.csv", sep=",")
train = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/sales_train.csv", sep=",")

In [124]:
train['date'] = pd.to_datetime(train['date'])

In [125]:
# df['day'] = df['date'].dt.day
# df['week'] = df['date'].dt.week
train['month'] = train['date'].dt.month

In [126]:
a = []
for i in train['date_block_num']:
    a.append(i+1)
    
g = []
for i in a:
    g.append(i%12)
h = []   
for i in g:
    if i ==0:
        h.append(12)
    else:
        h.append(i)
    

In [127]:
train['month'] = h

In [128]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month
0,2013-02-01,0,59,22154,999.00,1.0,1
1,2013-03-01,0,25,2552,899.00,1.0,1
2,2013-05-01,0,25,2552,899.00,-1.0,1
3,2013-06-01,0,25,2554,1709.05,1.0,1
4,2013-01-15,0,25,2555,1099.00,1.0,1
...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,10
2935845,2015-09-10,33,25,7460,299.00,1.0,10
2935846,2015-10-14,33,25,7459,349.00,1.0,10
2935847,2015-10-22,33,25,7440,299.00,1.0,10


In [129]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month
0,2013-02-01,0,59,22154,999.00,1.0,1
1,2013-03-01,0,25,2552,899.00,1.0,1
2,2013-05-01,0,25,2552,899.00,-1.0,1
3,2013-06-01,0,25,2554,1709.05,1.0,1
4,2013-01-15,0,25,2555,1099.00,1.0,1
...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,10
2935845,2015-09-10,33,25,7460,299.00,1.0,10
2935846,2015-10-14,33,25,7459,349.00,1.0,10
2935847,2015-10-22,33,25,7440,299.00,1.0,10


In [130]:
test['month'] = 11

In [131]:
test

,ID,shop_id,item_id,month
0,0,5,5037,11
1,1,5,5320,11
2,2,5,5233,11
3,3,5,5232,11
4,4,5,5268,11
...,...,...,...,...
214195,214195,45,18454,11
214196,214196,45,16188,11
214197,214197,45,15757,11
214198,214198,45,19648,11


In [132]:
possible = list(train['shop_id'].unique())
exists = test['shop_id'].tolist()

difference = pd.Series([item for item in possible if item not in exists])

target = test['shop_id'].append(pd.Series(difference))

target = target.reset_index(drop=True)

dummies = pd.get_dummies(
    target
)

dummieststshopid = dummies.drop(dummies.index[list(range(len(dummies)-len(difference), len(dummies)))])
dummiestrainshopid = pd.get_dummies(train['shop_id'])

In [133]:
dummiestrainshopid

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2935845,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2935846,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2935847,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [134]:
items_map = pd.Series(data=items['item_category_id'], index=items['item_id'])
train['item_id_category'] = train['item_id'].map(items_map)
test['item_id_category'] = test['item_id'].map(items_map)

In [135]:
# test['shop/itemidcat'] = test['shop_id']/test['item_id_category']
test['itemidcat/itemid'] = test['item_id_category']/test['item_id']
test['itemid/shopid'] = test['item_id']/test['shop_id']
a = []
for i in test['itemid/shopid']:
    if i == 0:
        a.append(0)   
    else:
        a.append(1000/i)
test['itemid/shopid'] = a

b = []
for i in test['item_id']:
    if i == 0:
        b.append(0)
    else:
        b.append(10000/i)
test['item_id'] = b



In [136]:
# train['shop/itemidcat'] = train['shop_id']/train['item_id_category']
train['itemidcat/itemid'] = train['item_id_category']/train['item_id']
train['itemid/shopid'] = train['item_id']/train['shop_id']


d = []
for i in train['item_id']:
    if i == 0:
        d.append(0)   
    else:
        d.append(1000/i)
train['item_id'] = d

c = []
for i in train['itemid/shopid']:
    if i == 0:
        c.append(0)   
    else:
        c.append(1000/i)
train['itemid/shopid'] = c

In [137]:
test

,ID,shop_id,item_id,month,item_id_category,itemidcat/itemid,itemid/shopid
0,0,5,1.985309,11,19,0.003772,0.992654
1,1,5,1.879699,11,55,0.010338,0.939850
2,2,5,1.910950,11,19,0.003631,0.955475
3,3,5,1.911315,11,23,0.004396,0.955657
4,4,5,1.898254,11,20,0.003797,0.949127
...,...,...,...,...,...,...,...
214195,214195,45,0.541888,11,55,0.002980,2.438496
214196,214196,45,0.617742,11,64,0.003954,2.779837
214197,214197,45,0.634639,11,55,0.003491,2.855874
214198,214198,45,0.508958,11,40,0.002036,2.290309


In [138]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,item_id_category,itemidcat/itemid,itemid/shopid
0,2013-02-01,0,59,0.045139,999.00,1.0,1,37,0.001670,2.663176
1,2013-03-01,0,25,0.391850,899.00,1.0,1,58,0.022727,9.796238
2,2013-05-01,0,25,0.391850,899.00,-1.0,1,58,0.022727,9.796238
3,2013-06-01,0,25,0.391543,1709.05,1.0,1,58,0.022709,9.788567
4,2013-01-15,0,25,0.391389,1099.00,1.0,1,56,0.021918,9.784736
...,...,...,...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,0.134971,299.00,1.0,10,55,0.007423,3.374275
2935845,2015-09-10,33,25,0.134048,299.00,1.0,10,55,0.007373,3.351206
2935846,2015-10-14,33,25,0.134066,349.00,1.0,10,55,0.007374,3.351656
2935847,2015-10-22,33,25,0.134409,299.00,1.0,10,57,0.007661,3.360215


In [139]:
train = pd.concat([train, dummiestrainshopid], axis = 1)
test = pd.concat([test, dummieststshopid], axis = 1)

In [140]:
submissionid = test['ID']
train = train.drop(['item_price','date','date_block_num','shop_id'],axis = 1)
test = test.drop(['shop_id','ID','shop_id'],axis = 1)
traini = train[:1000000]
validation = train[1000000:1700000]
Xval = validation.drop(['item_cnt_day'],axis = 1)
yval = validation.item_cnt_day
X = traini.drop(['item_cnt_day'],axis = 1)
y = traini.item_cnt_day

In [141]:
validation = train[1000000:1700000]

In [142]:
traini

,item_id,item_cnt_day,month,item_id_category,itemidcat/itemid,itemid/shopid,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0.045139,1.0,1,37,0.001670,2.663176,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.391850,1.0,1,58,0.022727,9.796238,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.391850,-1.0,1,58,0.022727,9.796238,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.391543,1.0,1,58,0.022709,9.788567,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.391389,1.0,1,56,0.021918,9.784736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.140905,1.0,10,29,0.004086,2.677188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999996,0.140924,1.0,10,30,0.004228,2.677565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999997,0.140924,2.0,10,30,0.004228,2.677565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999998,0.140924,1.0,10,30,0.004228,2.677565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [143]:
X.isnull().sum()

item_id             0
month               0
item_id_category    0
itemidcat/itemid    0
itemid/shopid       0
                   ..
55                  0
56                  0
57                  0
58                  0
59                  0
Length: 65, dtype: int64

In [144]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=random.randint(0,100))

In [145]:
# space={'max_depth': hp.quniform("max_depth", np.arange(1, 14, dtype=int)),
#         'gamma': hp.uniform ('gamma', 1,9),
#         'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
#         'reg_lambda' : hp.uniform('reg_lambda', 0,1),
#         'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
#         'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
#         'n_estimators': 180,
#         'seed': 0
#     }

In [ ]:
clf=xgb.XGBClassifier(max_depth= 5)
    
clf.fit(X_train, y_train,verbose=False)
    

#     accuracy = accuracy_score(y_test, pred>0.5)
#     print ("SCORE:", accuracy)
#     return {'loss': -accuracy, 'status': STATUS_OK }


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:24:08] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
pred = clf.predict(X_test)
predval = clf.predict(Xval)
error = mean_squared_error(y_test,pred)
error,mean_squared_error(yval,predval)

In [294]:
pred = clf.predict(X_test)
predval = clf.predict(Xval)
error = mean_squared_error(y_test,pred)
error,mean_squared_error(yval,predval)

(1.30605, 4.300616992012472)

In [121]:
pop = clf.predict(test)
subb = pd.DataFrame()
subb['ID'] = submissionid
subb['item_cnt_month'] = pop
subb.to_csv('firstoooinninixgboost4my.csv', index=False)
print('File Successfully Saved.!!!')

File Successfully Saved.!!!


In [23]:
possible = list(sales_train['item_id_category'].unique())
exists = test['item_id_category'].tolist()

difference = pd.Series([item for item in possible if item not in exists])

target = test['item_id_category'].append(pd.Series(difference))

target = target.reset_index(drop=True)

dummies = pd.get_dummies(
    target
)

dummieststitemid = dummies.drop(dummies.index[list(range(len(dummies)-len(difference), len(dummies)))])
dummieststitemidtrain = pd.get_dummies(sales_train['item_id_category'])

In [24]:
dummieststitemid

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
214196,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
214197,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
214198,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
dummieststitemidtrain

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2935845,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2935846,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2935847,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
